# <font color=darkcyan>  Kernel based regression and random forests </font>

### <font color=darkorange>  1. Independent simulated data</font>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
import pandas as pd

In [ ]:
# ignore warnings for better clarity (may not be the best thing to do)...
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(precision=2) 

In [ ]:
# function to sample a dataset - linear observation model
def sample_data_iid_linear(alpha,sigmay,n,d):
    X       = np.random.uniform(0,2*np.pi,[n,d])
    Y       = np.zeros(n)
    eta     = np.random.normal(loc=0,scale=1,size = n)
    Y[0]    = np.sum(np.cos(X[0,:])) + sigmay*eta[0]
    for k in range(1,n):
        Y[k]   = np.sum(alpha*X[k,:]) + sigmay*eta[k]
    return X, Y

In [ ]:
# function to sample a dataset - nonlinear observation model
def sample_data_iid_signal(sigmay,n,d):
    X       = np.random.uniform(0,2*np.pi,[n,d])
    Y       = np.zeros(n)
    eta     = np.random.normal(loc=0,scale=1,size = n)
    Y[0]    = np.sum(np.cos(X[0,:])) + sigmay*eta[0]
    for k in range(1,n):
        Y[k]   = np.sum(np.cos(X[k,:])) + sigmay*eta[k]
    return X, Y

In [ ]:
# set parameters
n_train = 1000
n_test = 100
d = 2
sigmay = 0.1 
true_alpha =  multivariate_normal(np.zeros(d), np.eye(d))
true_alpha

In [ ]:
# sample data  
X_train, Y_train = sample_data_iid_linear(true_alpha,sigmay,n_train,d)
X_test, Y_test = sample_data_iid_linear(true_alpha,sigmay,n_test,d)

#### <font color=darkred>  Linear regression for a simulated data set </font>

It is assumed that for all $1\leqslant i \leqslant n$, 

$$
Y_i = X^T_i \beta_{\star} + \varepsilon_i\,,
$$

where the $(\varepsilon_i)_{1\leqslant i\leqslant n}$ are i.i.d. random variables in $\mathbb{R}^d$, $X_i\in\mathbb{R}^d$ and $\beta_{\star}$ is an unknown vector in $\mathbb{R}^d$. Let $Y\in\mathbb{R}^d$ (resp. $\varepsilon\in\mathbb{R}^d$)  be the random vector such that  for all $1\leqslant i \leqslant n$, the $i$-th component of $Y$ (resp. $\varepsilon$) is $Y_i$ (resp. $\varepsilon_i$) and $X\in\mathbb{R}^{n\times d}$ the matrix with line $i$ equal to $X^T_i$. The model is then written

$$
Y = X \beta_{\star} + \varepsilon\,.
$$

In this section, it is assumed that $\mathbb{E}[\varepsilon] = 0$ and $\mathbb{E}[\varepsilon \varepsilon^T] = \sigma_{\star}^2 I_n$. The penalized least squares estimate of $\beta_{\star}$ is defined as a solution to

$$
\widehat \beta_n\in  \mathrm{argmin}_{\beta\in\mathbb{R}^d}\,\left( \|Y - X\beta\|_2^2 + \lambda \|\beta\|_2^2\right)\,.
$$

where $\lambda>0$.

<font color=darkred> Explain why the loss function is penalized (blackboard) </font>

The matrix $X^TX + \lambda I_n$ is definite positive for all $\lambda>0$ as for all $u\in\mathbb{R}^d$,

$$
u^T(X^TX + \lambda I_n)u = \|Xu\|_2^2 + \lambda \|u\|_2^2\,,
$$

which is positive for all $u\neq 0$. 

Therefore,  the matrix $X^TX + \lambda I_n$ is invertible for all $\lambda>0$. Using that for all $\beta>0$,

$$
\nabla \left(\|Y - X\beta\|_2^2 + \lambda \|\beta\|_2^2\right) = 2X^TX\beta - 2X^TY +  2\lambda\beta = 2\left\{\left(X^TX + \lambda I_d\right)\beta -X^TY\right\}\,.
$$

Therefore,

$$
\widehat \beta_n = \left(X^TX + \lambda I_d\right)^{-1}X^TY\,.
$$

<font color=darkred> Prove that the bias is given by (blackboard)
$$
\mathbb{E}[\widehat \beta_n] - \beta_* = - \lambda(X^TX + \lambda I_d)^{-1}\beta_*\,.
$$</font>

Note first that $\mathbb{E}[Y] = X\beta_*$ which yields

$$
\mathbb{E}[\widehat \beta_n] = \left(X^TX + \lambda I_d\right)^{-1}X^TX\beta_*
$$

and

$$
\mathbb{E}[\widehat \beta_n] - \beta_* = \left(X^TX + \lambda I_d\right)^{-1}X^TX\beta_* - \beta_* = \left(X^TX + \lambda I_d\right)^{-1}\left(X^TX - \left(X^TX + \lambda I_d\right)\right)\beta_* = - \lambda(X^TX + \lambda I_d)^{-1}\beta_*\,.
$$

<font color=darkred>Prove that the variance is given by (blackboard)
$$
\mathbb{V}[\widehat \beta_n] = \sigma_\star^2(X^TX + \lambda I_d)^{-2}X^TX\,.
$$</font>

By definition of $\widehat\beta_n$,

$$
\mathbb{V}[\widehat \beta_n] = \left(X^TX + \lambda I_d\right)^{-1}X^T\mathbb{V}[Y]X\left(X^TX + \lambda I_d\right)^{-1} = \sigma_*^2\left(X^TX + \lambda I_d\right)^{-1}X^TX\left(X^TX + \lambda I_d\right)^{-1}\,,
$$

since $\mathbb{V}[Y] = \sigma_*^2 I_n$. Let $\lambda_1,\ldots,\lambda_d\geqslant 0$ be the eignevalues of $X^TX$. Then, there exits an orthogonal matrix $P$, such that

$$
X^TX = P^{-1}\Delta P \quad\mathrm{and}\quad X^TX + \lambda I_d = P^{-1}\tilde\Delta P\,,
$$

where $\Delta = \mathrm{diag}(\lambda_1,\ldots,\lambda_d)$ and $\tilde\Delta = \mathrm{diag}(\lambda_1 + \lambda,\ldots,\lambda_d+\lambda)$.

$$
\left(X^TX + \lambda I_d\right)^{-1}X^TX\left(X^TX + \lambda I_d\right)^{-1} = P^{-1}\tilde\Delta^{-1} PP^{-1}\Delta PP^{-1}\tilde\Delta^{-1} P = P^{-1}\tilde\Delta^{-2}PP^{-1}\Delta P = \sigma_\star^2(X^TX + \lambda I_d)^{-2}X^TX\,.
$$

In [ ]:
linear_ridge_model = Ridge()
parameters = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1,1, 2, 3, 4, 5]}
ridge_regressor = GridSearchCV(linear_ridge_model, parameters,scoring='neg_mean_squared_error', cv=5)
ridge_regressor.fit(X_train, Y_train)
ridge_regressor.best_params_
ridge_regressor.best_score_

In [ ]:
# plot results
Y_pred_linear = ridge_regressor.predict(X_test)
plt.plot(Y_test,linestyle = "dashed",color="blue", label= "True observations")
plt.plot(Y_pred_linear,linestyle = "dashed",color="red", label="Predictions")
plt.ylabel('Observations', fontsize=12)
plt.xlabel('Individual indices', fontsize=12)
plt.tick_params(labelright=True)
plt.grid('True')
plt.legend();

In [ ]:
# sample data  
X_train, Y_train = sample_data_iid_signal(sigmay,n_train,d)
X_test, Y_test = sample_data_iid_signal(sigmay,n_test,d)

In [ ]:
linear_ridge_model = Ridge()
parameters = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1,1, 2, 3, 4, 5]}
ridge_regressor = GridSearchCV(linear_ridge_model, parameters,scoring='neg_mean_squared_error', cv=5)
ridge_regressor.fit(X_train, Y_train)
ridge_regressor.best_params_
ridge_regressor.best_score_
ridge_regressor.cv_results_

In [ ]:
# plot results
Y_pred_linear = ridge_regressor.predict(X_test)
plt.plot(Y_test,linestyle = "dashed",color="blue", label= "True observations")
plt.plot(Y_pred_linear,linestyle = "dashed",color="red", label="Predictions")
plt.ylabel('Observations', fontsize=12)
plt.xlabel('Individual indices', fontsize=12)
plt.tick_params(labelright=True)
plt.grid('True')
plt.legend();

#### <font color=darkred>  Kernel regression for a simulated data set </font>

A function $k:\mathbb{R}^d\times\mathbb{R}^d:\to \mathbb{R}$ is said to be a positive semi-definite kernel if and only if it is symmetric and if for all $n\geqslant 1$, $(x_1,\ldots,x_n)\in(\mathbb{R}^d)^n$ and all $(a_1,\ldots,a_n)\in\mathbb{R}^n$,
$$
\sum_{1\leqslant i,j\leqslant n}a_ia_jk(x_i,x_j) \geqslant 0\,.
$$

The following functions, defined on $\mathbb{R}^d\times\mathbb{R}^d$, are positive semi-definite kernels:

$$
k:(x,y)\mapsto x^Ty \quad\mathrm{and}\quad k:(x,y)\mapsto \mathrm{exp}\left(-\|x-y\|^2/(2\sigma^2\right)\,,\; \sigma>0\,.
$$


Let $\mathcal{F}$ be a Hilbert space of functions $f:\mathbb{R}^d\to\mathbb{R}$. A symmetric function $k:\mathbb{R}^d\times\mathbb{R}^d:\to \mathbb{R}$ is said to be a reproducing kernel of $\mathcal{F}$ if and only if:

1. for all $x\in\mathbb{R}^d$, $k(x,\cdot)\in\mathcal{F}$ ; 

2. for all $x\in\mathbb{R}^d$ and all $f\in\mathcal{F}$, $\langle f; k(x,\cdot)\rangle_\mathcal{F} = f(x)$ . 

The space $\mathcal{F}$ is said to be a reproducing kernel Hilbert space with kernel $k$.



Let $k:\mathbb{R}^d\times\mathbb{R}^d:\to \mathbb{R}$ be a positive definite kernel and $\mathcal{F}$ the RKHS with kernel $k$. Then (see previous session), 

$$
\widehat f^n_{\mathcal{F}} \in \underset{f\in\mathcal{F}}{\mathrm{min}}\;\frac{1}{n}\sum_{i=1}^n (Y_i - f(X_i))^2 + \lambda\|f\|_\mathcal{F}^2\,,
$$

where $\|f\|^2_\mathcal{F} = \langle f\,;\, f\rangle_\mathcal{F}$, is given by $\widehat f^n_{\mathcal{F}} : x \mapsto \sum_{i=1}^n \widehat \alpha_i k(X_i,x)$, where

$$
\widehat\alpha \in \underset{\alpha \in (\mathbb{R}^d)^n}{\mathrm{argmin}}\;\left\{\frac{1}{n}\|Y - K\alpha\|^2_2 + \lambda \sum_{1\leqslant i,j \leqslant n}\alpha_i \alpha_j k(X_i,X_j) = \frac{1}{n}\|Y - K\alpha\|^2_2 + \lambda \alpha^TK\alpha\right\}\,,
$$

where for all $1\leqslant i,j\leqslant n$, $K_{i,j} = k(X_i,X_j)$.

In practice, once the matrix $K$ is built, kernel ridge regression boils down to solving this optimization problem to obtain $\widehat \alpha$. Then, the estimated function $\widehat f^n_{\mathcal{F}}$ is a mixture of kernels evaluated at each data points with weights given by $\alpha$.

<font color=darkred> Provide the value of $\widehat \alpha$ when $K$ is invertible</font>

It is enough to follow the exact same steps as in the standard linear setting
$$
\nabla \left(\frac{1}{n}\|Y - K\alpha\|^2_2 + \lambda \alpha^TK\alpha\right) = \frac{2}{n}K^TK\alpha - \frac{2}{n}K^TY +  2\lambda\alpha = 2\left\{\left(\frac{1}{n}K^TK + \lambda I_n\right)\alpha - \frac{1}{n}K^TY\right\}\,.
$$

Therefore,

$$
\widehat \alpha = \frac{1}{n}\left(\frac{1}{n}K^TK + \lambda I_n\right)^{-1}K^TY\,.
$$

In practice, performing kernel regression amounts to choosing a kernel. Four common options are available: ``linear``,`` polynomial``, ``RBF`` and ``laplacian``. Then, obtaining the optimal parameters (the value of $\lambda$ and the parameters involved in the kernel function) may be done with ``GridSearchCV`` (see cell below) or ``RandomizedSearchCV`` (see random forests below). ``GridSearchCV`` performs an exhaustive search over the provided parameter values with cross-validation (based on ``cv`` number of folds) with ```scoring``` function to evaluate the predictions on the test set for each fold. In the case of ``RandomizedSearchCV``, not all parameter values are tested but only ``n_iter`` settings are sampled from specified values.

In [ ]:
kernel_ridge_regressor = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=5, 
                                      param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                                                  "gamma": np.logspace(-2, 2, 5)})
kernel_ridge_regressor.fit(X_train, Y_train)
kernel_ridge_regressor.best_params_
kernel_ridge_regressor.best_score_
kernel_ridge_regressor.cv_results_

In [ ]:
# plot results
Y_pred_kernel = kernel_ridge_regressor.predict(X_test)
plt.plot(Y_test,linestyle = "dashed",color="blue", label= "True observations")
plt.plot(Y_pred_kernel,linestyle = "dashed",color="red", label="Predictions")
plt.ylabel('Observations', fontsize=12)
plt.xlabel('Individual indices', fontsize=12)
plt.tick_params(labelright=True)
plt.grid('True')
plt.legend();

### <font color=darkorange> 2. Random forests for simulated time series</font>

Consider a Markov chain, given by $X_0 = x_0$ for $x_0\in \mathbb{R}^d$ and, for $k\geqslant 0$,

$$
X_{k+1} = \rho X_k + \sigma_X\varepsilon_k\,\,\, [2\pi]\,
$$ 

where $(\epsilon_k)_{k\geqslant 0}$ are i.i.d. standard Gaussian vectors in $\mathbb{R}^d$ $\sim {\sf N}(0,{\bf I}_d)$.

The observation model is

$$ 
Y_k = f(X_k) + \sigma_Y\eta_{k}\,,
$$

where $(\eta_k)_{k\geqslant 0}$ are i.i.d. $\sim N(0,1)$ and

$$
f:\begin{cases}
\mathbb{R}^d\to\mathbb{R}\\
{\bf x} \mapsto \sum_{i=1}^{d}\cos(x_i)
\end{cases}\,.
$$

The objective is to estimate the function $f$ using a training data set to predict the observations associated with the test dataset.

Start with a few imports, in particular the function ``RandomForestRegressor`` of sklearn that enables to apply the random forest algorithm in python.

In [ ]:
# ignore warnings for better clarity (may not be the best thing to do)...
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Define a function that takes as input the size $d$ of the vector $x_0$, the size $n$ of the dataset, the noise levels $\sigma_X$ of the autoregressive model and $\sigma_Y$ of the observed values and the parameter $\rho$ in the autoregressive model.

The function outputs data following the model described above.

In [ ]:
# function to sample a dataset 
def sample_data_ar(rho,sigmax,sigmay,n,d):
    X       = np.zeros(shape=(n,d))
    Y       = np.zeros(n)
    epsilon = np.random.normal(loc=0,scale=1,size = X.shape)
    eta     = np.random.normal(loc=0,scale=1,size = n)
    Y[0]    = np.sum(np.cos(X[0,:])) + sigmax*eta[0]
    for k in range(1,n):
        X[k,:] = (rho*X[k-1,:] + sigmax*epsilon[k,:])%(2*np.pi)
        Y[k]   = np.sum(np.cos(X[k,:])) + sigmay*eta[k]
    return X, Y

In [ ]:
# set parameters
n = 2000
d = 4

rho    = 0.1
sigmax = 0.1 
sigmay = 0.01 

Matrices X and Y containing data simulated according to the observation model described above may now be computed.

In [ ]:
# sample data  
X, Y = sample_data_ar(rho,sigmax,sigmay,n,d)

To evaluate different algorithms, the dataset is decomposed into training and test data.
In this type of time series analysis, the training data are the first values of $Y_k$ and the test are the last one. This corresponds to situations where one wants to predict future values of $Y$ given historical data.
This is somehow different form the i.i.d. case where taining and test data are chosen randomly using the function ``sklearn.model_selection.train_test_split``

In [ ]:
# split variables and observations, using 90% of the data set to estimate f 
import pandas as pd
df = pd.DataFrame(data = X, columns = ['X1', 'X2', 'X3', 'X4'])
df['Y'] = Y

nb_data_train = int(0.95*n)
nb_diff       = n-nb_data_train
df.head()
X_train = df.iloc[0:nb_data_train,:-1] 
X_test  = df.iloc[-nb_diff:,0:-1]
X_train.head()
Y_train = df.iloc[0:nb_data_train,-1] 
Y_test  = df.iloc[-nb_diff:,-1]

A first random forest prediction can be performed. 
``rf`` is the random forest function of sklearn when the number of trees in the forest is set to ``n\_trees``.
The forest is estimated using the training data ``X\_train`` and ``Y\_train`` and the values of $Y$ in the test set are compared to those predicted by the algorithm.
The mean-squared error between these quantities is also displayed.

In [ ]:
# number of trees in the forest for an elementary random forest estimate
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)

plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf,linestyle = "dashed", color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='red', linestyle = 'dashed', alpha = 0.5)
plt.xlabel('Test observations', fontsize=12)
plt.ylabel('Predicted observations', fontsize=12)
plt.tick_params(labelright=True)
plt.grid('True')

The predictions seem reasonable but it may be that these predictions are very accurate at first and then that the errors accumulate over time.
To test if this is the case, plot the values of $Y$ in the test set (true and predicted) as functions of the time from the last training time.

In [ ]:
plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future values','Random forest estimates'))

To improve this first prediction, the different parameters of the random forest algorithm may be tuned. For example, an increase of the number of trees in the forest reduced the varaince of the Monte Carlo estimate.

In [ ]:
# number of trees in the forest
n_trees = 500
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)


plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf,linestyle = "dashed", color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='red', linestyle = 'dashed', alpha = 0.5)
plt.xlabel('Test observations', fontsize=12)
plt.ylabel('Predicted observations', fontsize=12)
plt.tick_params(labelright=True)
plt.grid('True')

Note that this increases substancially the computational time although predictions seem equally relevant.

In [ ]:
plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future values','Random forest estimates'))

The objective is now to improve the algorithm using a cross-validation scheme to estimate the best value of these paramrters. It is useful to look at the parameters used by default in the algorithm here:
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Set a grid of parameters that will be tested by the cross-validation procedure.

In [ ]:
# with RandomizedSearchCVCreate, a grid of tuning parameters is built
# then a random search will be performed to test which parameter values yield the best 
# random forest estimate (see below).
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in the forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# max_features is the number of dimension considered to select the best split (the dimension along which a cell is cut)
max_features = ['log2', 'sqrt']
# The maximum depth of the tree corresponds to the maximum number of levels of the tree. 
# If not given, splits are performed until all cells contain less than min_samples_split samples.
max_depth = np.arange(5,100,5)
# min_samples_split is the minimum number of samples in a cell to allow a split.
min_samples_split = [2, 3, 4, 5, 6, 7, 8]
# A split is considered in the tree if it leaves at least min_samples_leaf training samples in each 
# subcell obtained after the spliting process.
min_samples_leaf = [1, 2, 3, 4]
# if bootstrap is true all the training dataset is used to build each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

Exploring all parameters combinations in this grid would be computationally prohibitive. 
An efficient alternative is to compare parameters chosen at random in the grid.

In [ ]:
rf = RandomForestRegressor()
# Random search among  all parameters (the number of possible combination is given. It is not a complete grid search !!!). 
# search across n_iter = 10 different combinations with a default 3-fold cross validation.
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=10, n_jobs = -1)

rf_random.fit(X_train,Y_train)

In [ ]:
# display the best parameters
rf_random.best_params_

In [ ]:
# build the best explored random forest
best_random = rf_random.best_estimator_
y_pred_rf = best_random.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)

In [ ]:
plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf,linestyle = "dashed", color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='red', linestyle = 'dashed', alpha = 0.5)
plt.xlabel('Test observations', fontsize=12)
plt.ylabel('Predicted observations', fontsize=12)
plt.tick_params(labelright=True)
plt.grid('True')

In [ ]:
# compute predictions usting test data and associated mean square error
y_pred_rf = rf_random.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)
plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future values','Random forest estimates'))

### <font color=darkorange> 3. Random forests for real data - Brazilian inflation prediction </font>

In [ ]:
# In this section, random forests are used to predic the Brazilian inflation based on
# many observed variables, see https://github.com/gabrielrvsc/HDeconometrics/
df = pd.read_csv('./BRinf')
df.head()

In [ ]:
# number of observations, number of variables
df.shape

The function ``pandas.DataFrame.corr`` may be used to compute the pairwise correlations between columns (variables and inflation). These correlations can be displayed using ``sns.heatmap`` to highlight highly correlated variables (and those likely to have an impact on the inflation).

See ``feature_importances_`` below for a first try at (relevant) variable selection.

In [ ]:
import seaborn as sns
corr = df.corr()
fig, ax = plt.subplots(figsize=(10,10))  
sns.heatmap(corr, xticklabels = False, yticklabels = False, cmap = 'Blues', ax = ax)

In [ ]:
corr.head()

In [ ]:
nb_data_train = 140
nb_diff       = df.shape[0]-nb_data_train
# inflation observations
Y_train = df.iloc[0:nb_data_train,1] 
Y_test  = df.iloc[-nb_diff:,1] 
Y_test.head()

In [ ]:
# other variables
X_train = df.iloc[0:nb_data_train,2:] 
X_test  = df.iloc[-nb_diff:,2:] 
X_test.head()

In [ ]:
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)

plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future inflation values','Random forest estimates'))

In [ ]:
# Many variables are used for the inflation prediction while very few observations are available.
# Selecting the most valuable variables is an alternativ to learn a simpler models
# This is obtained in Python with rf.feature_importances_

In [ ]:
plt.figure(1,figsize=(12,12))
plt.bar(list(df)[2:93],rf.feature_importances_,align='center')
plt.xticks(range(len(list(df)[2:93])),list(df)[2:93],rotation=90,size='small')
plt.title('Feature importances')

In [ ]:
# RandomForestClassifier and RandomForestRegressor use the gini importance mechanism as a measure of the 
# fetaures importance. The mean decrease in impurity importance of a 
# feature is computed by measuring t%%javascript
# see https://papers.nips.cc/paper/4928-understanding-variable-importances-in-forests-of-randomized-trees.pdf

In [ ]:
# collect the indices of the features with highest importance.
nb_features_to_keep = 10
ind = rf.feature_importances_.argsort()[-nb_features_to_keep:]
X_train.iloc[:,ind]

In [ ]:
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train.iloc[:,ind],Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test.iloc[:,ind])

plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future inflation values','Random forest estimates'))

<img src="./PCA_cor.png" width="500" height="700">

In [ ]:
# plot the MSE as a function of the numbers of variables used to estimate the function f
d_max = 60
MSE   = []
rf    = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)
for d in range(2,d_max): 
    rfd = RandomForestRegressor(n_estimators = n_trees)
    ind = rf.feature_importances_.argsort()[-d:]
    rfd.fit(X_train.iloc[:,ind],Y_train)
    # compute predictions usting test data and associated mean square error
    y_pred_rf = rfd.predict(X_test.iloc[:,ind])
    MSE = np.append(MSE,mean_squared_error(Y_test,y_pred_rf))

In [ ]:
#plt.figure(1,figsize=(8,8))
plt.plot(MSE,marker='o',color='k')
plt.xlabel('Number of features')
plt.ylabel('Mean square error on the test data set')

In [ ]:
# collect the indices of the features with highest importance.
nb_features_to_keep = 4
ind = rf.feature_importances_.argsort()[-nb_features_to_keep:]
X_train.iloc[:,ind]
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train.iloc[:,ind],Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test.iloc[:,ind])

plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future inflation values','Random forest estimates'))

In [ ]:
# you may add here a cross validation procedure using RandomizedSearchCV or GridSearchCV
# to select the best parameters (and compare the mean square error with what you obtain above).